In [1]:
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score
from pandas import ExcelWriter
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import numpy as np
import re
import random


def xy_sets(line):
    X=None
    y=None
    indexes = map(int, re.findall('\d+', line))
    aux = False
    for i in indexes:
        if aux is False:
            X = scaled[i, :]
            y = target[i]
            aux = True
            continue
        X = np.vstack((X, scaled[i, :]))
        y = np.hstack((y, target[i]))
        
    return X, y, indexes


multiplier = ['0,05', '0,01', '0,001']
n_hidden = [2, 3]
alpha = [0.001, 0.01, 0.1, 1]
momentum = [0.01, 0.1, 0.5]
dataset_name = ['57EDG', '54BND', 'combined']

for name in dataset_name:
    writer = ExcelWriter(name+'_ann.xlsx', engine='xlsxwriter')
    df = DataFrame()
    temp = DataFrame()
    for m in multiplier:
        for n in n_hidden:
            for a in alpha:
                for mo in momentum:
                    # load dataset
                    dataset = read_csv(name+'_'+m+'.csv', header=0, index_col=(0,1))
                    values = dataset.values

                    # split data set into data(input) and target(output)
                    data = values[:, 1:] 
                    target = values[:, 0]

                    # normalize features
                    scaled = normalize(data[:, :3], norm='max', axis=0)
                    scaled = np.concatenate((scaled, data[:, :4]), axis = 1)

                    scores = np.zeros(20)
                    col = np.array([])
                    cont = 0
                    sensitivity = 0  # true positives rate
                    specificity = 0  # true negatives rate
                    y_test_full = np.array([])
                    y_cont_full = np.array([])

                    file = open(name+'_kf.txt', 'r')
                    while cont < 20:

                        train_line = file.readline()
                        test_line = file.readline()

                        X_train, y_train, train_indexes = xy_sets(train_line)
                        X_test, y_test, test_indexes = xy_sets(test_line)
                        
                        classifier = MLPClassifier(hidden_layer_sizes=(n,), solver='lbfgs', 
                                                    alpha=a, random_state=11, verbose=True, momentum=mo)  # classifier 
                        classifier.fit(X_train, y_train)
                        scores[cont] = classifier.score(X_test, y_test)
                        y_pred = classifier.predict(X_test)

                        y_test_full = np.hstack((y_test_full, y_test))
                        y_cont_full = np.hstack((y_cont_full, y_pred))

                        cont += 1

                    file.close()

                    print(confusion_matrix(y_test_full, y_cont_full))

                    acc = accuracy_score(y_test_full, y_cont_full)
                    recall = recall_score(y_test_full, y_cont_full, average=None)
                    sns = recall[0]
                    spf = recall[1]
                    
                    col = np.hstack((col, a))
                    col = np.hstack((col, mo))
                    col = np.hstack((col, m))
                    col = np.hstack((col, scores))
                    col = np.hstack((col, acc))
                    col = np.hstack((col, sns))
                    col = np.hstack((col, spf))
                    # col = np.hstack((col, std))

                    df.insert(0, n, col, True)
                    row_na = df.index[0]
                    df = df.rename(index={row_na: 'alpha'})
                    row_na = df.index[1]
                    df = df.rename(index={row_na: 'momentum'})
                    row_na = df.index[2]
                    df = df.rename(index={row_na: 'polygonal_model'})
                    
                    row_na = df.index[-1]
                    df = df.rename(index={row_na: 'specificity'})
                    row_na = df.index[-2]
                    df = df.rename(index={row_na: 'sensibility'})
                    row_na = df.index[-3]
                    df = df.rename(index={row_na: 'accuracy'})
                    
                    print(scores.mean())
                    print("Accuracy: %0.2f" % (acc))
                    print("Sensitivity: %0.2f" % (sns))
                    print("Specificity: %0.2f" % (spf))

    df.to_excel(writer, sheet_name=name)
    writer.save()
    writer.close()
    

[[33  4]
 [12  8]]
0.725
Accuracy: 0.72
Sensitivity: 0.89
Specificity: 0.40
[[33  4]
 [12  8]]
0.725
Accuracy: 0.72
Sensitivity: 0.89
Specificity: 0.40
[[33  4]
 [12  8]]
0.725
Accuracy: 0.72
Sensitivity: 0.89
Specificity: 0.40
[[34  3]
 [12  8]]
0.7416666666666667
Accuracy: 0.74
Sensitivity: 0.92
Specificity: 0.40
[[34  3]
 [12  8]]
0.7416666666666667
Accuracy: 0.74
Sensitivity: 0.92
Specificity: 0.40
[[34  3]
 [12  8]]
0.7416666666666667
Accuracy: 0.74
Sensitivity: 0.92
Specificity: 0.40
[[34  3]
 [12  8]]
0.7416666666666667
Accuracy: 0.74
Sensitivity: 0.92
Specificity: 0.40
[[34  3]
 [12  8]]
0.7416666666666667
Accuracy: 0.74
Sensitivity: 0.92
Specificity: 0.40
[[34  3]
 [12  8]]
0.7416666666666667
Accuracy: 0.74
Sensitivity: 0.92
Specificity: 0.40
[[35  2]
 [12  8]]
0.7583333333333333
Accuracy: 0.75
Sensitivity: 0.95
Specificity: 0.40
[[35  2]
 [12  8]]
0.7583333333333333
Accuracy: 0.75
Sensitivity: 0.95
Specificity: 0.40
[[35  2]
 [12  8]]
0.7583333333333333
Accuracy: 0.75
Sensiti

[[14 14]
 [ 3 23]]
0.675
Accuracy: 0.69
Sensitivity: 0.50
Specificity: 0.88
[[14 14]
 [ 3 23]]
0.675
Accuracy: 0.69
Sensitivity: 0.50
Specificity: 0.88
[[14 14]
 [ 3 23]]
0.675
Accuracy: 0.69
Sensitivity: 0.50
Specificity: 0.88
[[14 14]
 [ 3 23]]
0.675
Accuracy: 0.69
Sensitivity: 0.50
Specificity: 0.88
[[14 14]
 [ 3 23]]
0.675
Accuracy: 0.69
Sensitivity: 0.50
Specificity: 0.88
[[14 14]
 [ 3 23]]
0.675
Accuracy: 0.69
Sensitivity: 0.50
Specificity: 0.88
[[15 13]
 [ 2 24]]
0.7166666666666666
Accuracy: 0.72
Sensitivity: 0.54
Specificity: 0.92
[[15 13]
 [ 2 24]]
0.7166666666666666
Accuracy: 0.72
Sensitivity: 0.54
Specificity: 0.92
[[15 13]
 [ 2 24]]
0.7166666666666666
Accuracy: 0.72
Sensitivity: 0.54
Specificity: 0.92
[[17 11]
 [ 5 21]]
0.6833333333333333
Accuracy: 0.70
Sensitivity: 0.61
Specificity: 0.81
[[17 11]
 [ 5 21]]
0.6833333333333333
Accuracy: 0.70
Sensitivity: 0.61
Specificity: 0.81
[[17 11]
 [ 5 21]]
0.6833333333333333
Accuracy: 0.70
Sensitivity: 0.61
Specificity: 0.81
[[22  6]
 

[[49 16]
 [ 4 42]]
0.825
Accuracy: 0.82
Sensitivity: 0.75
Specificity: 0.91
[[49 16]
 [ 4 42]]
0.825
Accuracy: 0.82
Sensitivity: 0.75
Specificity: 0.91
[[49 16]
 [ 4 42]]
0.825
Accuracy: 0.82
Sensitivity: 0.75
Specificity: 0.91
[[50 15]
 [ 4 42]]
0.8333333333333334
Accuracy: 0.83
Sensitivity: 0.77
Specificity: 0.91
[[50 15]
 [ 4 42]]
0.8333333333333334
Accuracy: 0.83
Sensitivity: 0.77
Specificity: 0.91
[[50 15]
 [ 4 42]]
0.8333333333333334
Accuracy: 0.83
Sensitivity: 0.77
Specificity: 0.91
[[51 14]
 [ 4 42]]
0.8433333333333334
Accuracy: 0.84
Sensitivity: 0.78
Specificity: 0.91
[[51 14]
 [ 4 42]]
0.8433333333333334
Accuracy: 0.84
Sensitivity: 0.78
Specificity: 0.91
[[51 14]
 [ 4 42]]
0.8433333333333334
Accuracy: 0.84
Sensitivity: 0.78
Specificity: 0.91
[[53 12]
 [ 7 39]]
0.8333333333333334
Accuracy: 0.83
Sensitivity: 0.82
Specificity: 0.85
[[53 12]
 [ 7 39]]
0.8333333333333334
Accuracy: 0.83
Sensitivity: 0.82
Specificity: 0.85
[[53 12]
 [ 7 39]]
0.8333333333333334
Accuracy: 0.83
Sensiti